In [1]:
import json
import numpy as np

In [2]:
import os
import sys
import argparse
import torch

sys.path.append(os.getcwd())

# import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive

import json
import random

import string
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  

In [3]:
s = "string. With. Punctuation?" # Sample string 
s.translate(str.maketrans('', '', string.punctuation))

'string With Punctuation'

In [4]:
example_sent = "This is a sample sentence, showing off the stop words filtration."
stop_words = set(stopwords.words('english')) 
word_tokens = word_tokenize(example_sent) 
filtered_sentence = [w for w in word_tokens if not w in stop_words] 
  

In [5]:
# DUMP_FILE = '../../data/personachat_self_original_comet.json'
DUMP_FILE = '../../data/personachat_self_original_comet_validation.json'  # ** VALIDATION SPLIT ONLY

In [6]:
def load_data(dump_fname):
    annotated_data = json.load(open(dump_fname,'r'))
    return annotated_data

In [7]:
annotated_data = load_data(DUMP_FILE)

In [8]:
annotated_data.keys()

dict_keys(['valid'])

In [9]:
split = 'valid'

In [10]:
len(annotated_data), len(annotated_data[split]), annotated_data[split][0].keys()

(1, 1000, dict_keys(['personality', 'utterances', 'coment_annotation']))

In [11]:
# annotated_data['train']

In [12]:
annotated_data[split][0]['coment_annotation']

[{'sent': 'i read twenty books a year .',
  'comet': {'oEffect': {'event': 'i read twenty books a year .',
    'effect_type': 'oEffect',
    'beams': ['none',
     'they get a good grade',
     'wants to read more',
     'they learn about the books',
     'they learn about the book']},
   'oReact': {'event': 'i read twenty books a year .',
    'effect_type': 'oReact',
    'beams': ['none', 'happy', 'interested', 'impressed', 'proud']},
   'oWant': {'event': 'i read twenty books a year .',
    'effect_type': 'oWant',
    'beams': ['none',
     'to teach them more',
     'to read more books',
     'to read more',
     'to teach them']},
   'xAttr': {'event': 'i read twenty books a year .',
    'effect_type': 'xAttr',
    'beams': ['intelligent', 'studious', 'intellectual', 'smart', 'educated']},
   'xEffect': {'event': 'i read twenty books a year .',
    'effect_type': 'xEffect',
    'beams': ['gains knowledge',
     'learns a lot',
     'learns something new',
     'none',
     'learns 

In [13]:
len(annotated_data[split][0]['utterances']), annotated_data[split][0]['utterances'][0]

(8,
 {'candidates': ['oh really ? i am actually in high school and i am graduating as class of 2019 !',
   "that's an interesting choice . i'd have to pick french fries",
   'i just got a pet fish for my 18th birthday yesterday from my parents .',
   'yeah , well what about you ?',
   'my favorite watch is the rolex ? what is yours ?',
   "what is in spain that's so interesting",
   "i don't like clowns . they are scary to a kid like me",
   'poetry . roses are red . violet are . . . ?',
   'my father is a member of the army , served for 10 years now .',
   'oh i like mexican food , but my favorite food are cheeseburgers',
   'hey there , are you a mother ?',
   "it sure is . i'd like to see more of the city though .",
   'it is not so fun i have 2 friend who speak a different langues',
   "i'd like some honey though . do you sell it ?",
   'i am a recovering heavy drinker . full time . how about you ?',
   'hi ! i have three kids . how many do you have ?',
   'awesome ! i own 2 dogs ,

In [14]:
annotated_data[split][0]['utterances'][1]

{'candidates': ['why have you not sent help ? ! the scorpions are stinging my legs ! ree ! ! ! ! ! ! !',
  'that is great i am expecting twins in two months . will these be your first kids ?',
  'do you live on a farm or ranch ?',
  'hi how are you doing tonight i am fine .',
  "i'd love to see her do that .",
  "i don't . but i am so glad you do something that brings you joy",
  'it is hard , buy my dog keeps me company . do you have dogs ?',
  'sounds like a good plan , what would you like to teach ?',
  'i like rap music and i also produce for music artists .',
  'where do you work ?',
  'i broke my arm so i can not drink coffee',
  "i meant mickey . cool i've a lot of friends and love the playground . do you ?",
  'oh , yes . i wish i could skate to school . i ride the bus instead .',
  'that is my idea of heaven . i love bunnies ! i donate to a bunny charity too .',
  'i work on a freelance basis as an author , blogger and affiliate marketer . and you ?',
  'like me ? would you go

In [15]:
annotated_data[split][0]['utterances'][-1]

{'candidates': ['fun , i should take my kids out to do that . do you have a family ?',
  'they were popular in my younger days lol',
  'i sing and love rock especially while working . keeps me pumped .',
  'i mean where i come from kitties do not play with birds .',
  'i tend my rose garden and read books . i am deaf in one ear',
  'you can learn to say something nice and smart',
  'when in europe all i do is shop',
  'wow , that is exciting ! i never went to a nascar race .',
  'i live in hawaii and i work from home .',
  'hello jennifer , i am a proud dad and husband',
  'and my spouse used all the money i earned',
  'oh young people . the holidays are coming . that makes me sad .',
  'i had surgery last week and now i think of an object , and it moves !',
  'hello ! i just got back from a run . what are you up to today ?',
  'hello , my name is ally and my favourite color is orange',
  "no i didn't see that . i am excited for the new star wars though",
  'that is exciting ! do you g

In [16]:
def process_text(s):
    s = s.lower()
    s = s.translate(str.maketrans('', '', string.punctuation))
    word_tokens = word_tokenize(s) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return filtered_sentence
    

In [29]:
def process(item):
    
    utterances = item['utterances']
    history = utterances[-1]['history']
    all_conversation_words = []
    for h in history:
        #print("history: ", h)
        all_conversation_words.extend(process_text(h))
    all_conversation_words = set(all_conversation_words)

    personality = item['personality']
    all_personality_words = []
    for h in personality:
        #print("personality: ", h)
        all_personality_words.extend(process_text(h))
    all_personality_words = set(all_personality_words)
    
    coment_annotation = item['coment_annotation']
    all_comet_words = []
    for h in coment_annotation:
        comet = h['comet']
        for _,value in comet.items():
            for sent in value['beams']:
                #print('coment_annotation: ',sent)
                if sent.strip() == 'none':
                    continue
                all_comet_words.extend(process_text(sent))
    all_comet_words = set(all_comet_words)
    
    
    conv_personality_overlap = len(all_conversation_words.intersection(all_personality_words)) / ( len(all_conversation_words) )
    conv_comet_overlap = len(all_conversation_words.intersection(all_comet_words)) /  ( len(all_conversation_words) )
    
    all_extended_personality_words = all_comet_words.union(all_personality_words)
    conv_ext_personality_overlap = len(all_conversation_words.intersection(all_extended_personality_words)) /  ( len(all_conversation_words) )
    
    additional_word_set = all_conversation_words.intersection(all_comet_words) - all_conversation_words.intersection(all_personality_words)
    
    #print("additional_word_set = ", additional_word_set)
    #print(conv_personality_overlap, conv_comet_overlap)
    
    return {'additional_word_set':additional_word_set, 
            'conv_personality_overlap':conv_personality_overlap,
            'conv_comet_overlap':conv_comet_overlap,
            'additional_recall':conv_ext_personality_overlap - conv_personality_overlap}
    

In [30]:
process(annotated_data[split][0])

{'additional_word_set': {'done', 'get', 'good', 'help', 'well', 'work'},
 'conv_personality_overlap': 0.1111111111111111,
 'conv_comet_overlap': 0.15873015873015872,
 'additional_recall': 0.09523809523809523}

In [ ]:
all_ret_split = []
for datum in annotated_data[split]:
    all_ret_split.append(process(datum))

In [ ]:
len(all_ret_split)

In [ ]:
additional_recall_vals_split = [val['additional_recall'] for val in all_ret_split]

In [ ]:
import scipy.stats

In [ ]:
scipy.stats.describe(additional_recall_vals_split)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(additional_recall_vals_split)